### **Summarization: An Unsupervised Extractive Way**

###### **There are two types of approach for text summarization: 1) Supervised and 2) Unsupervised. And in terms of summaries, there are two types of summaries: 1) Extractive and 2) Abstractive.<br>Here, we will use a Unsupervised extractive method.**

#### **Steps**

##### **1. Generate Embedding Model**
###### **Embedding selects sentences that contain words with the same meaning as the most relevant words — even if the words are different. So, before even preprocessing the text, the program must define the embedding model. To obtain the word embedding, I split the text into words and passed it to Word2vec.** 

##### **2. Preprocessing**
###### **Preprocessing includes splitting the text into sentences, lowering the case of all words, removing stopwords (is, an, the, etc.) and punctuation and other tasks. Especifically to identifying and splitting the text into sentences, the algorithm will score and select them to be part of the summary.**

##### **3. Find Most Relevant Words Embedding Representation**
###### **The algorithm uses TF-IDF to find the most relevant words of the text. These words are the centroid of the text. After finding the words centroid, the program sums up the vectors of the words that are part of the centroid, and that sum is the embedding representation of the centroid.**

##### **4. Score Sentences**
###### **The sentences are scored based on how similar they are to the centroid embedding. In order to compare to the centroid embedding, the algorithm calculates the embedding representation of each sentence.**
###### ***`Sentence embedding = sum of words vectors that are part of the sentence.`***
###### **Lastly, after the sentence embedding is defined, the algorithm uses cosine similarity to calculate the similarity between the centroid and the sentence embedding. Each sentence gets a score based on how similar they are to the centroid.**

##### **5. Select Sentences and Resolve Redundancy**
###### **The sentences are selected based on their score. The number of sentences selected is limited by how many words the summary should contain (50 words, 100 words, or?). A common problem when dealing with automatic summarization is handling redundancy — too similar sentences being part of the summary. To overcome that, when selecting the sentences, you compare them with the sentences already in summary. If a chosen sentence is too similar to one in summary, you don’t add it to the final text. The algorithm uses cosine similarity with some predefined threshold when making the comparison.**